# Evolver Loop 1 Analysis

Analyzing the baseline and identifying improvement opportunities.

In [ ]:
import pandas as pd
import numpy as np
import os
import json

# Tree geometry (15 vertices)
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_vertices(x, y, deg):
    """Get the vertices of a tree at position (x, y) with rotation deg."""
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotated_x = TX * cos_a - TY * sin_a + x
    rotated_y = TX * sin_a + TY * cos_a + y
    return rotated_x, rotated_y

def get_bounding_box(trees_df):
    """Calculate the bounding box side length for a set of trees."""
    all_x = []
    all_y = []
    
    for _, row in trees_df.iterrows():
        x = float(str(row['x'])[1:]) if str(row['x']).startswith('s') else float(row['x'])
        y = float(str(row['y'])[1:]) if str(row['y']).startswith('s') else float(row['y'])
        deg = float(str(row['deg'])[1:]) if str(row['deg']).startswith('s') else float(row['deg'])
        
        vx, vy = get_tree_vertices(x, y, deg)
        all_x.extend(vx)
        all_y.extend(vy)
    
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)
    side = max(max_x - min_x, max_y - min_y)
    return side

def calculate_score(df):
    """Calculate the total score for a submission."""
    scores = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        trees = df[df['id'].str.startswith(prefix)]
        if len(trees) != n:
            continue
        side = get_bounding_box(trees)
        contribution = (side ** 2) / n
        scores.append({'n': n, 'side': side, 'contribution': contribution})
    scores_df = pd.DataFrame(scores)
    total_score = scores_df['contribution'].sum()
    return total_score, scores_df

print('Functions defined')

In [ ]:
# Check N=1 configuration in the baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(baseline_path)

# Get N=1 configuration
n1_trees = df[df['id'].str.startswith('001_')]
print('N=1 configuration:')
print(n1_trees)

# Parse the values
x = float(str(n1_trees.iloc[0]['x'])[1:])
y = float(str(n1_trees.iloc[0]['y'])[1:])
deg = float(str(n1_trees.iloc[0]['deg'])[1:])
print(f'\nPosition: ({x:.6f}, {y:.6f}), Angle: {deg:.2f} degrees')

In [ ]:
# Calculate bounding box for N=1 at different angles
def get_single_tree_bbox(deg):
    """Get bounding box side for a single tree at given angle."""
    vx, vy = get_tree_vertices(0, 0, deg)
    width = max(vx) - min(vx)
    height = max(vy) - min(vy)
    return max(width, height)

# Test angles from 0 to 90 degrees
angles = np.linspace(0, 90, 91)
bbox_sizes = [get_single_tree_bbox(a) for a in angles]

best_angle = angles[np.argmin(bbox_sizes)]
best_bbox = min(bbox_sizes)

print(f'Current N=1 angle: {deg:.2f} degrees, bbox: {get_single_tree_bbox(deg):.6f}')
print(f'Best angle: {best_angle:.2f} degrees, bbox: {best_bbox:.6f}')
print(f'Improvement potential: {get_single_tree_bbox(deg) - best_bbox:.6f}')

In [ ]:
# Compare multiple pre-optimized solutions
import glob

preopt_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/'
csv_files = [
    preopt_dir + 'santa-2025.csv',
    preopt_dir + 'bucket-of-chump/submission.csv',
    preopt_dir + 'best_ensemble.csv',
    preopt_dir + 'ensemble.csv',
]

results = []
for csv_file in csv_files:
    if os.path.exists(csv_file):
        try:
            df = pd.read_csv(csv_file)
            score, _ = calculate_score(df)
            results.append({'file': os.path.basename(csv_file), 'score': score})
            print(f'{os.path.basename(csv_file)}: {score:.6f}')
        except Exception as e:
            print(f'{os.path.basename(csv_file)}: Error - {e}')

In [ ]:
# Check santa25-public versions
public_dir = preopt_dir + 'santa25-public/'
for f in sorted(os.listdir(public_dir)):
    if f.endswith('.csv'):
        try:
            df = pd.read_csv(public_dir + f)
            score, _ = calculate_score(df)
            results.append({'file': f, 'score': score})
            print(f'{f}: {score:.6f}')
        except Exception as e:
            print(f'{f}: Error - {e}')

In [ ]:
# Find the best solution
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('score')
print('\nBest solutions:')
print(results_df.head(10))

In [ ]:
# Analyze score breakdown for the best solution
best_file = preopt_dir + 'santa-2025.csv'
df = pd.read_csv(best_file)
total_score, scores_df = calculate_score(df)

print(f'Total score: {total_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total_score - 68.919154:.6f}')

# Score by N range
print('\nScore breakdown by N range:')
for start in [1, 11, 51, 101, 151]:
    end = min(start + 49, 200)
    if start == 1:
        end = 10
    elif start == 11:
        end = 50
    range_score = scores_df[(scores_df['n'] >= start) & (scores_df['n'] <= end)]['contribution'].sum()
    print(f'  N={start}-{end}: {range_score:.4f}')

In [ ]:
# Analyze which N values have the most room for improvement
# Theoretical minimum for N trees is when they're packed perfectly
# For a single tree, the minimum bounding box is achieved at 45 degrees

# Calculate efficiency for each N
scores_df['efficiency'] = scores_df['n'] / (scores_df['side'] ** 2)
print('\nLowest efficiency N values (most room for improvement):')
print(scores_df.nsmallest(20, 'efficiency')[['n', 'side', 'contribution', 'efficiency']])

In [ ]:
# Check if N=1 is at 45 degrees
n1 = df[df['id'].str.startswith('001_')]
deg_n1 = float(str(n1.iloc[0]['deg'])[1:])
print(f'N=1 angle: {deg_n1:.2f} degrees')
print(f'Is it at 45 degrees? {abs(deg_n1 - 45) < 1}')

# Calculate the score contribution at 45 degrees vs current
current_side = get_single_tree_bbox(deg_n1)
optimal_side = get_single_tree_bbox(45)

print(f'\nCurrent N=1 side: {current_side:.6f}, contribution: {current_side**2:.6f}')
print(f'Optimal N=1 side (45 deg): {optimal_side:.6f}, contribution: {optimal_side**2:.6f}')
print(f'Potential improvement: {current_side**2 - optimal_side**2:.6f}')